In [1]:
# Import Statements 
####################
import os 
import sys
import copy
import pandas as pd
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import pathlib
import datetime
import itertools as ite 
import math
import calendar

NUM_TIME_BINS_PER_DAY = 24

In [ ]:
_="""

Define any useful functions 

"""

def cleanURL(url):
    p = pathlib.Path(url)
    path = str(p.as_posix()) 
    return path 


def getDF(loc, sheetname):
    dataframe = pd.read_excel(loc, sheetname)
    #https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe
    dataframe = dataframe.applymap(lambda x: x.strip() if type(x) is str else x)
    return dataframe

def printNulls(df):
    null_columns = df.columns[df.isnull().any()]
    return df[null_columns].isnull().sum() 


def writeDFToFile(dfs, path_): #dfs is an array of dataframes and their sheet names , path needs to have
    time_ = str(datetime.datetime.now())
    current_date_time = time_[0:time_.index(".")]
    current_date_time = current_date_time.replace(":", "-")
    task4_fileoutput = path_+current_date_time+".xlsx"

    writer = pd.ExcelWriter(task4_fileoutput)
    
    for df_tuple in dfs:  
        df = df_tuple[0]
        sheetName = df_tuple[1]
        df.to_excel(writer, sheetName)
    print("file written to :       " + task4_fileoutput)
    writer.save()



In [ ]:
##############
##############
# Seans Stuff Before 11/27/2018 dataset
##############
##############




# In[3]:


###################
# LOAD CRIME DATA #
###################
crime_dataframe = pd.read_csv(cleanURL(r'C:\Users\j70514\Documents\Data Science Stuff\DeepLearning_cs230\CNN_data_crunch/Crimes (Chicago).csv'))
crbk = crime_dataframe.copy()
crime_dataframe = crime_dataframe.head(100000).copy()

# In[4]:


########################
# CONDITION CRIME DATA #
########################
# Delete columns that are redundant or unhelpful
columns_to_delete = ['Case Number', 'Location Description', 'Block', 'Arrest', 'Domestic', 'FBI Code', 'IUCR', 'Description','X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Location']
final_dataframe = crime_dataframe.drop(columns_to_delete, axis = 1).copy().dropna() ## changed
print('Length: %d' % len(final_dataframe))


# In[5]:


##################################
# CONVERT CRIME STATS TO ONE-HOT #
##################################
# Convert 'Beat', 'District', 'Ward', and 'Community Area' to one-hot vectors
final_dataframe['Beat'] = final_dataframe['Beat'].astype(int)#.astype(str).apply(lambda x: 'BEAT_'+x)
final_dataframe['District'] = final_dataframe['District'].astype(int)#.astype(str).apply(lambda x: 'DISTRICT_'+x)
final_dataframe['Ward'] = final_dataframe['Ward'].astype(int)#.astype(str).apply(lambda x: 'WARD_'+x)
final_dataframe['Community Area'] = final_dataframe['Community Area'].astype(int)#.astype(str).apply(lambda x: 'COMMUNITY_'+x)
print('Length: %d' % len(final_dataframe))


# In[7]:


##################
# CONDITION DATE #
##################
# Convert crime dates to YEAR, MONTH, DAY, HOUR, MINUTE, and weekday columns
# Convert those to one-hot and concat with final dataframe
final_dataframe['Date'] = pd.to_datetime(final_dataframe['Date'])    ### modified 
final_dataframe['YEAR'] = final_dataframe['Date'].dt.year#.astype(str).apply(lambda x: 'YEAR_'+x)
final_dataframe['MONTH'] = final_dataframe['Date'].dt.month#.apply(lambda x: calendar.month_abbr[x])
final_dataframe['DAY'] = final_dataframe['Date'].dt.day#.astype(str).apply(lambda x: 'DAY_'+x)
final_dataframe['WEEKDAY'] = final_dataframe['Date'].dt.weekday.apply(lambda x: calendar.day_name[x])
hours = final_dataframe['Date'].dt.hour
minutes = final_dataframe['Date'].dt.minute
final_dataframe['TIME_OF_DAY'] = ((hours + minutes / 60.) / 24. * NUM_TIME_BINS_PER_DAY).astype(int)#.astype(str).apply(lambda x: 'TIME_SLOT_'+x)
print('Length: %d' % len(final_dataframe))


# In[12]:


####################
# JOIN TEMPERATURE #
####################
temperature_dataframe = pd.read_csv(cleanURL(r'C:\Users\j70514\Documents\Data Science Stuff\DeepLearning_cs230\CNN_data_crunch/Temperatures (Chicago).csv'))
# Drop the TAVG column because it has too many NaNs
temperature_dataframe = temperature_dataframe.drop(['TAVG'], axis = 1)  ## modified 
# Convert the Precipitation, max T, and min T columns to float
temperature_dataframe['PRCP'] = pd.to_numeric(temperature_dataframe['PRCP'])
temperature_dataframe['TMAX'] = pd.to_numeric(temperature_dataframe['TMAX'])
temperature_dataframe['TMIN'] = pd.to_numeric(temperature_dataframe['TMIN'])
temperature_dataframe.rename(columns={'PRCP':'PRECIPITATION', 'TMAX':'MAX TEMP', 'TMIN':'MIN TEMP'}, inplace = True)
# Join with the final dataframe
temperature_dataframe['date_join'] = pd.to_datetime(temperature_dataframe['DATE']).dt.date
final_dataframe['date_join'] = final_dataframe['Date'].dt.date
final_dataframe = final_dataframe.merge(temperature_dataframe, on=['date_join'], how='left').drop(['DATE', 'STATION', 'NAME', 'date_join'], axis = 1).dropna()## modified 
print('Length: %d' % len(final_dataframe))


In [ ]:
#Save final_dataframe for reference
final_dataframe.to_csv(cleanURL(r'C:\Users\j70514\Documents\Data Science Stuff\DeepLearning_cs230\CNN_data_crunch/buildOff3.csv') , sep = ',' )


# Really, after we finalize input dataset
# retrieve final_dataframe and start from here
#final_dataframe  = pd.read_csv()

In [ ]:
############
# Step 1.Assume Latitude and Longitude are normally distributed, filter out anything +/- 2 sigmas for 95% of data - empirically  not many datapoints are removed
############  
#get the old values 
latMax1 = final_dataframe.Latitude.max()
latMin1 = final_dataframe.Latitude.min()
longMax1  = final_dataframe.Longitude.max()
longMin1  = final_dataframe.Longitude.min()
print(latMax1 - latMin1)
print(longMax1 - longMin1)

#get mean and standard deviations to use for filteration
latMean = final_dataframe.Latitude.mean()
longMean = final_dataframe.Longitude.mean()
latStd = final_dataframe.Latitude.std()  #we could toggle degrees of freedom but for now I don't think it matters 
longStd = final_dataframe.Longitude.std()

# perform filteration 
final_dataframe = final_dataframe.loc[(final_dataframe['Latitude'] < latMean+2*latStd) & (final_dataframe['Latitude'] > latMean-2*latStd)].copy()
final_dataframe = final_dataframe.loc[(final_dataframe['Longitude'] < longMean+2*longStd) & (final_dataframe['Longitude'] > longMean-2*longStd)].copy()

latMax2 = final_dataframe.Latitude.max()
latMin2 = final_dataframe.Latitude.min()
longMax2  = final_dataframe.Longitude.max()
longMin2  = final_dataframe.Longitude.min()
print(latMax2 - latMin2)
print(longMax2 - longMin2)



In [ ]:
############
# Step 2. Generate Image Size and Scaling Multipliers to convert to image format. Have a 256 by 256 datasize. 
############

iHeight = 256.
iWidth = 256.

# pixel/degree ratios
latMultiplier = (iHeight- 0.)/(latMax2 - latMin2)  
longMultiplier = (iWidth- 0.)/(longMax2 - longMin2)

#convert the latitude and longitude values to the pixel values
final_dataframe['latPixel'] = (final_dataframe.Latitude - latMin2)*latMultiplier
final_dataframe['longPixel'] =(final_dataframe.Longitude - longMin2)*longMultiplier

#pixel values are floats and not integers. type cast
final_dataframe['latPixel'] = final_dataframe['latPixel'].astype(np.int64)
final_dataframe['longPixel'] = final_dataframe['longPixel'].astype(np.int64)


In [ ]:
############
# Step 3. Provide masks for precincts, beats, district, and wards
############
#Kludge


In [ ]:
###########
# Step 4. Get ready for the Data Generation Loop 
###########


#initialize values for the for loop 
mindate = final_dataframe.Date.min().date()
maxdate = final_dataframe.Date.max().date()

delta = maxdate - mindate

#https://stackoverflow.com/questions/7274267/print-all-day-dates-between-two-dates
datesInDays = [ mindate + datetime.timedelta(index) for index in range(delta.days+1)] 
timeOfDays = [i for i in range(NUM_TIME_BINS_PER_DAY)]


#we want the cartesian product of days and the time of days -> this is how many images we are making
dates_and_timeOfDays_iterator = ite.product(datesInDays, timeOfDays)
dates_and_timeOfDays = [[z[0], z[1]] for z in dates_and_timeOfDays_iterator]  # has the day and the time of day

#check we got all possible combinations 
assert(len(dates_and_timeOfDays) == NUM_TIME_BINS_PER_DAY * len(datesInDays))

#https://stackoverflow.com/questions/13635032/what-is-the-inverse-function-of-zip-in-python
#allDates, allTimesOfDay  = zip(*dates_and_timeOfDays) # we unzip the cartesian product so that we can now loop through everything

In [ ]:
###########
# Step 5. The For loop for generating the images 
###########


#The General Algorithm:
# For each day:
#   For each time of day:
#       dataframe = get the data points for this time period
#       generate an image


# For faster filtering, remove some columns
all_cols = final_dataframe.columns.tolist()
all_cols.remove('ID')
all_cols.remove('Latitude') # have the pixelated values already
all_cols.remove('Longitude')
fDF = final_dataframe[all_cols].copy()

#assert datatypes
assert(final_dataframe['TIME_OF_DAY'].dtype == np.dtype('int32'))
assert(final_dataframe['DAY'].dtype == np.dtype('int64'))
assert(final_dataframe['MONTH'].dtype == np.dtype('int64'))
assert(final_dataframe['YEAR'].dtype == np.dtype('int64'))


LAYERS_IN_A_IMAGE = 5

images_x = np.zeros(shape = (len(dates_and_timeOfDays), LAYERS_IN_A_IMAGE , int(iHeight),int(iWidth)))
outputs_y = np.zeros(10) 

outputs = []

dt = None
tofD = None 
zero_img = np.zeros(shape = (int(iHeight),int(iWidth)))



In [ ]:
## Step 5 continued... the actual for loop for adding data

for i, _ in enumerate(dates_and_timeOfDays):
    dt = _[0]
    tofD = _[1]
    df = fDF.loc[(tofD == fDF['TIME_OF_DAY']) & (dt.day == fDF['DAY']) & (dt.month == fDF['MONTH']) & (dt.year == fDF['YEAR'])]

    img = np.zeros((LAYERS_IN_A_IMAGE,zero_img.shape[0], zero_img.shape[1]))
    #the lat/long base layer
    #img[0] = np.array(zero_img)
    if(len(df) != 0):
        row = df.latPixel.values
        col = df.longPixel.values
        img[0][row,col] = 1.
        #to check:  a = np.where(img ==1), a[0] , a[1]
    
    #temperature layers 
    # Max temperature 
    img[1] = df['MAX TEMP'].iat[0]
    # Min temperature 
    img[2] =  df['MIN TEMP'].iat[0]
    # PRECIPITATION
    img[3] =  df['PRECIPITATION'].iat[0]
    
    # TODO: add more layers here

    images_x[i] = img 

In [ ]:


#######
# Save data

np.save(r'/Volumes/GoogleDrive/My Drive/Crime Data/Composite Data/Sean Workspace/27_November_cnn.npy', images_x)
np.save(r'/Volumes/GoogleDrive/My Drive/Crime Data/Composite Data/Sean Workspace/27_November_yOutput.npy', outputs_y)



